# Linear sorting

All sorting utilizing the comparison model are lower bounded by $O(nlogn)$. The direct access arrays have $O(1)$ read and write times. Therefore by leveraging direct access arrays sorting can be performed in $O(n)$. 

The rest of notebook goes through the developement of a sorting algorithm (radix sort) that achieves $O(n)$ sorting performance. 

## Direct Access Sort

This data structure requires unique keys. It instantiates a direct access array of length $u$. Where is the max range of values of the keys can take. The keys are taken from the set $[0,u]$. 

Fist a direct access array of length $u$ needs to be instantiated costing $O(u)$.

Secondly, the items need to be inserted into the direct access array. Each insert operation run in constant time resulting in a total cost of $O(n)$ to insert all items.

Then the direct access array needs to be iterated through yeilding its stored value if present. This returns a sorted version of the list and runs in $O(u)$.

Cons:
- The issues with direct access array is that it assumes unique keys, otherwise collision exists for the direct access array. 
- Moreover it works well only when $u=O(n)$, otherwise sorting complexity is more than $O(n)$ 

In [1]:
# %load -s direct_access_sort modules/linear_sort.py
def direct_access_sort(A):
    u=1+max([a.key for a in A]) #O(n)
    D=[None]*u #O(u)
    for a in A: #O(n)
        D[a.key]=a
    i=0
    for d in D: #O(u)
        if d is not None:
            A[i]=d
            i+=1


In [2]:
a_key=[2,4,9,3,6,12]
A=[type('Node', (object,), {'key': key}) for key in a_key]
print([a.key for a in A])
direct_access_sort(A)
print([a.key for a in A])

[2, 4, 9, 3, 6, 12]
[2, 3, 4, 6, 9, 12]


## Counting Sort 

Conting sort solves the issue of not being able to sort lists with duplicate keys. To do so at each index in the direct access array another data structure is used to store the keys. This would help in the event of collisions arising from duplicate keys. For radix sort it is important that the order of the duplicate keys is the same for the output from sorting as it is for the input (i.e stable sorting). To do so a sequence data structure needs to be used at each index of the direct access array. In the implementation below a python list is used to do so. 

The rest of the operations involved in the sorting is the same as the direct access array. 

In [3]:
# %load -s counting_sort modules/linear_sort.py
def counting_sort(A):
    u=1+max([a.key for a in A]) #0(n)
    D=[[] for _ in range(u)] #O(u)
    for a in A: #O(n)
        D[a.key].append(a)

    i=0
    for chain in D: #O(u)
        for x in chain:
            A[i]=x
            i+=1


In [4]:
a_key=[2,4,9,3,9,6,12]
A=[type('Node', (object,), {'key': key}) for key in a_key]
print([a.key for a in A])
counting_sort(A)
print([a.key for a in A])

[2, 4, 9, 3, 9, 6, 12]
[2, 3, 4, 6, 9, 9, 12]


## Radix Sort
The only con that exist from direct access sort mentioned above is the fact that the sorting is $O(u)$. In which $O(u)$ can be worser than $O(nlogn)$ if the space of keys is large, causing preference to the usage of merge sort instead. However, radix sorts exists to allow for a wider range of the key space $u$ while still maintaing a $O(n)$ time sorting algorithm. It does so by performing tuple sort with the counting sort algorithm. 

If for instance $O(u)=O(n^c)$ where $c$ is a constant, you can represent the keys as tuples with each elements of the tuple being the representation of $u$ in base $n$. Then the list can be sorted by performing tuples sort on the tuple from the least significant base to the most significant.


Summary
- As long as $O(u)=O(n^c)$ where $c$ is a constant, radix sort can sort items in $O(n)$ 
- It can handle repeated keys like counting sort
- It can't handle negative keys but keys can be manupulated to do so. 

In [5]:
# %load -s radix_sort modules/linear_sort.py
def radix_sort(A):
    u = 1 + max([a.key for a in A])
    n = len(A)
    c = 1 + u.bit_length()//n.bit_length()

    D=[]
    class Obj: pass

    for i in range(n): #O(cn)
        D.append(Obj())
        D[i].item=A[i]
        D[i].digits=[]
        high=A[i].key
        for j in range(c):
            high,low=divmod(high,n)
            D[i].digits.append(low)
    
    for i in range(c): #O(cn)
        for j in range(n):
            D[j].key=D[j].digits[i]
        counting_sort(D)
    
    for i,d in enumerate(D): #O(n)
        A[i]=d.item 

    return A


In [6]:
a_key=[2,4,3,1,9,14,4,9,3,6,12]
A=[type('Node', (object,), {'key': key}) for key in a_key]
print([a.key for a in A])
radix_sort(A)
print([a.key for a in A])

[2, 4, 3, 1, 9, 14, 4, 9, 3, 6, 12]
[1, 2, 3, 3, 4, 4, 6, 9, 9, 12, 14]


# Promlem set 3

## Problem 3-5 Anagram Archaelogy

String A is an anagram of another string B if A is a permutation of the letters in B; for example,
(indicatory, dictionary) and (brush, shrub) are pairs of words that are anagrams of
each other. In this problem, all strings will be ASCII strings containing only the lowercase English
letters a to z.

Given two strings A and B, the anagram substring count of B in A is the number of contiguous
substrings of A that are anagrams of B. For example, if A = ’esleastealaslatet’ and B =
’tesla’, then, of the 13 contiguous substrings in A of length |B| = 5, exactly 3 of them are
anagrams of B, namely (’least’, ’steal’, ’slate’), so the anagram substring count of
B in A is 3. 

##### (a) Given string A and a positive integer k, describe a data structure that can be built in O(|A|) time, which will then support a single operation: given a different string B with |B| = k, return the anagram substring count of B in A in O(k) time. 

Acknowledging that all anagrams have the same frequency count of lower case characters the following algorithm is proposed:

1) As you loop through $A$ build a frequency count of each character for the $n-k$ contiguous substrings, where $|A| = n$. Build a hashmap to keep track of the counts of each contiguous substring. Use the frequency count representation for each contiguous substring as keys for mapping with a hash map. $-> O(n)$
2) Build a frequency count of each character for the string $B$. Using this frequency count index the hashmap to see if an anagram of string $B$ exists in $A$. If yes return the count stored in the hashmap, if no return zero. $-> O(k)$


##### (b)  Given string T and an array of n length-k strings S = (S0, . . . , Sn−1) satisfying 0 < k < |T |, describe an O(|T | + nk)-time algorithm to return an array A = (a0, . . . , an−1) for which ai is the anagram substring count of Si in T for all i ∈ {0, . . . , n − 1}. 

Based on the answer to part a the following algorithm is proposed.

1) As you loop through $T$ build a frequency count of each character for the $|T|-k$ contiguous substrings. Build a hashmap to keep track of the counts of each contiguous substring. Use the frequency count representation for each contiguous substring as keys for mapping with a hash map. $-> O(|T|)$

2) For each string $S_{i}$  within $S$  build a frequency count of each character. Using this frequency count index the hashmap to see if an anagram of string $S_{i}$ exists in $T$. If yes append $A$ with the count stored in the hashmap, if append with zero. $-> O(nk)$

##### (c)  Write a Python function count anagram substrings(T, S) that implements your algorithm from part (b). Note the built-in Python function ord(c) returns the ASCII integer corresponding to ASCII character c in O(1) time. 

```python
def count_anagram_substrings(T, S):
    '''
    Input:  T | String
            S | Tuple of strings S_i of equal length k < |T|
    Output: A | Tuple of integers a_i:
              | the anagram substring count of S_i in T
    '''
    A = []
    ##################
    char_index=lambda c: ord(c) - ord('a')
    k = len(S[0])
    anagram_dict = {}

    #stores the running count of characters in the current substring of T
    t_dict = [0 for _ in range(26)] 
    
    for i,t in enumerate(T):
        t_dict[char_index(t)]+=1

        if i>=k-1:
            if i>=k:
                t_dict[char_index(T[i-k])]-=1
            
            key = tuple(t_dict)

            if key in anagram_dict:
                anagram_dict[key]+=1
            else:
                anagram_dict[key]=1

    for S_i in S:
        #stores the count of characters in the current Si substring
        s_dict = [0 for _ in range(26)]
        for s in S_i:
            s_dict[char_index(s)]+=1
        
        key = tuple(s_dict)
        if key in anagram_dict:
            A.append(anagram_dict[tuple(s_dict)])
        else:
            A.append(0)
    
    ##################
    return tuple(A)

```
 